# Lab 6 - Genetic Algorithms

In [274]:
import numpy as np
import matplotlib.pyplot as plt

<h2>Task 1:</h2> Write a function in Python that takes a decision array x, a weight array w, a value array
v, a limit on weight W, a penalty multiplier r and an exponent β, and returns the result of
φstatic(x, f(·), g(·)) 

In [275]:
#returns the objective function value of a solution
def phi_static(x_arr, w_arr, v_arr, f, g, r, beta, w_lim):
    # x_arr: decision array
    # f: objective function
    # g: constraint function
    # r: penalty parameter
    # beta: exponent
    # W: constraint limit
    return f(x_arr, v_arr) - r * max(0, g(x_arr, w_arr, w_lim)) ** beta

#returns the total weight of a solution
def g(x_arr, w_arr, w_lim):
    return np.sum(x_arr * w_arr) - w_lim

#returns the total cost of a solution
def f(x_arr, v_arr):
    return np.sum(x_arr * v_arr)

In [276]:
data = np.array([
    (1, 'Banana', 600, 1.35),
    (2, 'Raspberries', 180, 2.25),
    (3, 'Blueberries', 250, 2.5),
    (4, 'Medjool Dates', 500, 4.50),
    (5, 'Pineapple', 400, 1.75),
    (6, 'Mango', 250, 2.5),
    (7, 'Watermelon', 380, 2),
    (8, 'Strawberries', 400, 3.5)
], dtype=[('id', 'i4'), ('name', 'U20'), ('weight', 'i4'), ('value', 'f4')])

print(data)
print(phi_static(np.array([1, 1, 1, 1, 1, 0, 0, 0]), data['weight'], data['value'], f, g, 0.00001, 2, 2000))
print(phi_static(np.array([1, 1, 1, 1, 1, 1, 0, 0]), data['weight'], data['value'], f, g, 0.00001, 3, 2000))
print(phi_static(np.array([1, 1, 1, 1, 1, 1, 1, 0]), data['weight'], data['value'], f, g, 0.00001, 3, 2000))

[(1, 'Banana', 600, 1.35) (2, 'Raspberries', 180, 2.25)
 (3, 'Blueberries', 250, 2.5 ) (4, 'Medjool Dates', 500, 4.5 )
 (5, 'Pineapple', 400, 1.75) (6, 'Mango', 250, 2.5 )
 (7, 'Watermelon', 380, 2.  ) (8, 'Strawberries', 400, 3.5 )]
12.350000023841858
-43.46999997615815
-1739.3099999761582


<h2>Task 2:</h2>
Implement a function for generating a random population (as a 2-dimensional binary array).
This function should take the number of rows and number of columns as parameters.

In [277]:
def gen_rand_pop (rows, cols):
    return np.random.randint(2, size=(rows, cols))
print(gen_rand_pop(10, 8))

[[1 1 0 0 1 1 1 1]
 [0 0 0 0 0 0 0 1]
 [1 1 1 1 1 0 1 0]
 [0 0 0 0 1 0 0 1]
 [1 0 0 1 0 0 0 1]
 [0 1 1 1 0 1 0 0]
 [1 1 0 1 1 1 0 1]
 [0 0 0 1 1 0 1 1]
 [0 1 1 1 0 0 0 0]
 [1 0 0 1 1 0 0 1]]


<h2>Task 3:</h2>
Implement the truncation selection function with k = 4

In [278]:
def trunc_selec(data, pop_num, w_lim, k):
    # pop: population
    # phi: objective function
    # w_lim: weight limit
    # k: number of selected individuals
    # returns: selected individuals
    pop = gen_rand_pop(pop_num, 8)
    
    for i in range(pop_num):
        pop[i][7] = phi_static(pop[i], data['weight'], data['value'], f, g, 0.001, 3, w_lim)
        
    #sort by objective function value from highest to lowest
    pop = pop[pop[:,7].argsort()[::-1]]

    #return the k best individuals
    return pop[:k]
    

In [279]:
print(trunc_selec(data, 10, 2000, 5))

[[ 0  0  1  0  1  1  1 12]
 [ 1  0  1  1  0  0  0 11]
 [ 0  0  1  0  1  1  0 10]
 [ 0  1  1  0  1  0  0 10]
 [ 1  1  1  0  0  0  0  9]]


<h2>Task 4 & 5:</h2>Implement the one-point crossover function and bitwise mutation function

In [280]:
def one_point_crossover(parent1, parent2, probability):
    # parent1: first parent
    # parent2: second parent
    # returns: two children
    child1 = [0, 0, 0, 0, 0, 0, 0, 0]
    child2 = [0, 0, 0, 0, 0, 0, 0, 0]
    crossover_point = np.random.randint(1, 7) 
    
    for i in range(crossover_point):
        if np.random.random() < probability:
            child1[i] = parent2[i]
            child2[i] = parent1[i]
        
    for i in range(crossover_point, 8):
        if np.random.random() < probability:
            child1[i] = parent1[i]
            child2[i] = parent2[i]
        
    return child1, child2 

def bitwise_mutation(child, p):
    # child: child to mutate
    # p: probability of mutation
    # returns: mutated child
    for i in range(8):
        if np.random.random() < p:
            child[i] = 1 - child[i]
    return child

In [281]:
# create a test array of 8 random binary numbers
test_arr = np.random.randint(2, size=8)
test_arr2 = np.random.randint(2, size=8)
print(test_arr)
print(test_arr2)
print(one_point_crossover(test_arr, test_arr2, 0.5))
print(bitwise_mutation(test_arr, 0.1))


[0 0 0 0 0 0 0 0]
[1 1 0 0 1 0 0 1]
([0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1])
[0 0 0 0 1 0 0 0]


testing

In [282]:
best = trunc_selec(data, 10, 2500, 4)
print(best)

# take a solution and output the names from the data
def get_data(x, data):
    data_arr = []
    for i in range(len(x)):
        if x[i] == 1:
            data_arr.append(data[i])
    return data_arr

# take a population and output the names from the data
def get_data_pop(pop, data): 
    pop_arr = []
    for x in pop:
        pop_arr.append(get_data(x, data))
    return pop_arr

best_arr = get_data_pop(best, data)

for x in best_arr:
    print(x)
    #calculate total weight in the solution
    total_weight = 0
    total_price = 0
    for y in x:
        total_weight += y[2]
        total_price += y[3]
    total_price = round(total_price, 2)
    print("Total weight: ", total_weight, "Total price: ", total_price)


[[ 1  1  1  0  1  0  1 13]
 [ 0  1  1  1  1  0  1 13]
 [ 0  1  1  1  0  0  1 11]
 [ 1  1  1  0  0  0  0  9]]
[(1, 'Banana', 600, 1.35), (2, 'Raspberries', 180, 2.25), (3, 'Blueberries', 250, 2.5), (5, 'Pineapple', 400, 1.75), (7, 'Watermelon', 380, 2.)]
Total weight:  1810 Total price:  9.85
[(2, 'Raspberries', 180, 2.25), (3, 'Blueberries', 250, 2.5), (4, 'Medjool Dates', 500, 4.5), (5, 'Pineapple', 400, 1.75), (7, 'Watermelon', 380, 2.)]
Total weight:  1710 Total price:  13.0
[(2, 'Raspberries', 180, 2.25), (3, 'Blueberries', 250, 2.5), (4, 'Medjool Dates', 500, 4.5), (7, 'Watermelon', 380, 2.)]
Total weight:  1310 Total price:  11.25
[(1, 'Banana', 600, 1.35), (2, 'Raspberries', 180, 2.25), (3, 'Blueberries', 250, 2.5)]
Total weight:  1030 Total price:  6.1


<h2>Task 5</h2>Implement the Genetic Algorithm discussed in the lecture

In [283]:
def genetic_algo(data, pop_size, w_lim, its, cross_prob, mut_prob):
    # data: data array
    # w_lim: weight limit
    # pop_size: population size
    # p: probability of mutation
    # its: number of iterations
    # returns: best solution
    
    #use truncation selection to generate an initial population
    pop = trunc_selec(data, pop_size, w_lim, pop_size)
    
    for i in range(its):
        #select the top 2 individuals from the population as parents
        parent1 = pop[0]
        parent2 = pop[1]
        
        #crossover the parents
        child1, child2 = one_point_crossover(parent1, parent2, cross_prob)
        
        #mutate the children
        child1 = bitwise_mutation(child1, mut_prob)
        child2 = bitwise_mutation(child2, mut_prob)
        
        #replace the worst 2 individuals with the children
        pop[-1] = child1
        pop[-2] = child2
        
        #calculate the objective function value for each individual
        for j in range(pop_size):
            pop[j][7] = phi_static(pop[j], data['weight'], data['value'], f, g, 0.00001, 3, w_lim)
            
        #sort the population by objective function value, highest to lowest
        pop = pop[pop[:,7].argsort()[::-1]]
        print(pop)
        
    return pop[0]


In [284]:
best = genetic_algo(data, 10, 2500, 10, 0.8, 0.2)
print(best)
#print the names of the items in the best solution
best_arr = get_data(best, data)
print(best_arr)
total_weight = 0
total_price = 0
for x in best_arr:
    total_weight += x[2]
    total_price += x[3]
total_price = round(total_price, 2)
print("Total weight: ", total_weight, "Total price: ", total_price)

[[    0     0     1     1     1     0     0 18911]
 [    0     0     0     1     1     1     0 18911]
 [    1     1     0     0     0     0     1 13454]
 [    0     0     1     1     0     1     0  8921]
 [    0     1     1     0     0     0     0    18]
 [    0     0     0     1     0     0     0    18]
 [    1     1     1     0     1     1     0   -31]
 [    0     1     0     1     1     0     1 -3235]
 [    0     0     1     0     0     0     1 -3501]
 [    0     0     0     1     1     0     0 -5092]]
[[     1      0      0      0      1      1      0  83906]
 [     1      0      0      1      0      0      0  67040]
 [     0      0      0      1      1      1      0  61800]
 [     0      0      1      1      1      0      0  61800]
 [     1      1      0      0      0      0      1  51425]
 [     0      0      1      1      0      1      0  18883]
 [     0      0      0      1      0      0      0  11326]
 [     1      1      1      0      1      1      0    -98]
 [     0      1  